In [2]:
import os, sys 
import pandas as pd
import numpy as np
os.chdir('../src')
import config
import glob
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import random
from sklearn.metrics import accuracy_score
import joblib
from sklearn.cluster import KMeans
from scipy.spatial.distance import cosine
import IPython.display as ipd

In [3]:
nn_folder = 'stacker_xgb_7'
sub = {}
for flag in ['train', 'test']:
    _ = pd.read_csv(config.OUTPUT_FOLDER + '%s/sub_%s.csv'%(nn_folder, flag))
    _.rename(columns = {'target' : 'pred_target'}, inplace = True)
    _['confidence'] = _[config.mapping_dict.keys()].max(axis = 1)
    gt = pd.read_csv(config.DATA_FOLDER + 'gt_%s_stats.csv'%flag)
    _ = _.merge(gt, on = 'id')
    sub[flag] = _
    assert len(_) == len(gt)
feature_columns = config.mapping_dict.keys()

In [134]:
N = 1194
def get_item(df_):
    items = df_[(df_['confidence'] >= 0.519373893738) & (~df_['pred_target'].isin(config.allowed_train_labels + ['silence']))]
#  df_[(df_['pred_target'].isin(config.allowed_train_labels))]
    audio_id = items['id'].values[N]
    confidence = items['confidence'].values[N]
    volume = items['stat_max_audio'].values[N]
    print('Predicted', items['pred_target'].values[N], confidence, audio_id)
    item = items[feature_columns].values[N]
    return item, audio_id
item, audio_id = get_item(sub['test'])
ipd.Audio(config.DATA_FOLDER + 'test/audio/%s.wav'%audio_id) # load a local WAV file

('Predicted', 'zero', 0.51950061321300001, 'clip_fc093a0b3')


In [142]:
for c in [x for x in config.mapping_dict.keys() if x not in  config.allowed_train_labels] + ['silence']:
    zz =  df_[df_['pred_target'] == c].copy()
    N = np.clip(int(0.2 * len(zz)), 0,1000)
    print(c,N)
    zz = zz.sort_values('confidence', ascending = False).head(N)

('sheila', 298)
('seven', 1000)
('house', 232)
('one', 1000)
('four', 1000)
('zero', 1000)
('wow', 305)
('six', 1000)
('three', 1000)
('bird', 629)
('happy', 309)
('marvin', 530)
('eight', 1000)
('two', 1000)
('tree', 146)
('dog', 219)
('bed', 305)
('cat', 217)
('nine', 1000)
('five', 1000)
('silence', 1000)


In [143]:
zz

,id,pred_target,bed,bird,cat,dog,down,eight,five,four,...,up,wow,yes,zero,confidence,path,stat_max_audio,stat_min_audio,stat_mean_audio,stat_std_audio
158530,clip_d187c0c5f,silence,0.016186,0.016207,0.016198,0.016209,0.016206,0.016188,0.016192,0.016208,...,0.016204,0.016183,0.016170,0.016196,0.514041,/home/tony/ssd_buffer/kaggle_speech/test/audio...,12760,-14537,-19.760250,1891.403545
22755,clip_f99ecf625,silence,0.016186,0.016207,0.016198,0.016209,0.016206,0.016188,0.016192,0.016208,...,0.016204,0.016183,0.016170,0.016196,0.514041,/home/tony/ssd_buffer/kaggle_speech/test/audio...,12317,-13685,-15.747875,1978.705514
102956,clip_3a817701a,silence,0.016186,0.016207,0.016198,0.016209,0.016206,0.016188,0.016192,0.016208,...,0.016204,0.016183,0.016170,0.016196,0.514041,/home/tony/ssd_buffer/kaggle_speech/test/audio...,13533,-12508,17.253500,1994.324267
56652,clip_8c4633fee,silence,0.016186,0.016207,0.016198,0.016209,0.016206,0.016188,0.016192,0.016208,...,0.016204,0.016183,0.016170,0.016196,0.514041,/home/tony/ssd_buffer/kaggle_speech/test/audio...,9835,-13132,0.766188,1773.172723
138526,clip_ff3eb9c5f,silence,0.016186,0.016207,0.016198,0.016209,0.016206,0.016188,0.016192,0.016208,...,0.016204,0.016183,0.016170,0.016196,0.514041,/home/tony/ssd_buffer/kaggle_speech/test/audio...,15371,-13345,7.907938,1853.544996
102353,clip_df88dcd17,silence,0.016186,0.016207,0.016198,0.016209,0.016206,0.016188,0.016192,0.016208,...,0.016204,0.016183,0.016170,0.016196,0.514041,/home/tony/ssd_buffer/kaggle_speech/test/audio...,13355,-11454,20.301813,1826.693505
101257,clip_cf16bdc3b,silence,0.016186,0.016207,0.016198,0.016209,0.016206,0.016188,0.016192,0.016208,...,0.016204,0.016183,0.016170,0.016196,0.514041,/home/tony/ssd_buffer/kaggle_speech/test/audio...,13355,-12694,30.747375,1837.371811
139182,clip_8918767fb,silence,0.016186,0.016207,0.016198,0.016209,0.016206,0.016188,0.016192,0.016208,...,0.016199,0.016183,0.016170,0.016196,0.514041,/home/tony/ssd_buffer/kaggle_speech/test/audio...,15371,-13345,9.449438,1851.095405
100099,clip_b2f81b4b6,silence,0.016186,0.016207,0.016198,0.016209,0.016206,0.016188,0.016192,0.016208,...,0.016204,0.016183,0.016170,0.016196,0.514041,/home/tony/ssd_buffer/kaggle_speech/test/audio...,13003,-13132,-1.678250,1794.114940
59114,clip_9df2b872d,silence,0.016186,0.016207,0.016198,0.016209,0.016206,0.016188,0.016192,0.016208,...,0.016204,0.016183,0.016170,0.016196,0.514041,/home/tony/ssd_buffer/kaggle_speech/test/audio...,11783,-14745,-17.282625,1803.384281


In [135]:
df_['pred_target'].value_counts(zz)

silence    12111
four       11561
zero        9136
up          7614
nine        7459
on          6694
five        6582
one         6468
left        6418
no          6216
off         6062
seven       5846
go          5725
three       5669
eight       5635
two         5624
down        5619
right       5571
six         5526
yes         5526
stop        5511
bird        3145
marvin      2651
happy       1546
bed         1525
wow         1525
sheila      1491
house       1160
dog         1099
cat         1089
tree         734
Name: pred_target, dtype: int64

0.519373893738

In [94]:
df_ = sub['test']
df_[(df_['confidence'] >= 0.519373893738) & (~df_['pred_target'].isin())]['pred_target'].value_counts()

zero     3698
eight    2345
Name: pred_target, dtype: int64